In [13]:
import json
import urllib
import requests
import pandas as pd
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import re

In [14]:
url = 'https://thegreatestbooks.org/?p='
page = requests.get(url).text
display(page)

'<!DOCTYPE html>\n<html>\n  <head>\n    <title>The Greatest Books: The Best Books - 1 to 50</title>\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n      <meta name="description" content="The Top 50 greatest fiction books of all time determined by 128 lists and articles from various critics, authors and experts." />\n    <link rel="stylesheet" media="all" href="/assets/application-7e9b0ca4311cbd5d92d9c8ffe873320c4075f373ec54cb88cf1a59cc04895e0c.css" />\n    <meta name="csrf-param" content="authenticity_token" />\n<meta name="csrf-token" content="rj/XtR7ZVaTi1aEXiiNU6IgIV4oxb2M6g0X0eFgM5MYdlDK5zYwGOxJRWmK1E0tVoBmBbugQ991EakBVhNitYA==" />\n    <!-- HTML5 Shim and Respond.js IE8 support of HTML5 elements and media queries -->\n    <!-- WARNING: Respond.js doesn\'t work if you view the page via file:// -->\n    <!--[if lt IE 9]>\n      <script src="https://oss.maxcdn.com/libs/html5shiv/3.7.0/html5shiv.js"></script>\n      <script src="https://oss.maxcdn.com/lib

In [22]:
url = 'https://thegreatestbooks.org/?p='

main_books = []

html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')
books = soup.find_all('div', {'class': 'container'})
main_books.append(books)
soup.select('li.item:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > h4:nth-child(1)')

[<h4>
             1
 . <a href="/items/225">In Search of Lost Time </a> by <a href="/authors/4798">Marcel Proust</a>
 </h4>]

In [21]:
for heading in soup.find_all(re.compile("^h[4]")):
    print(heading.name + ' ' + heading.text.strip())

h4 1
. In Search of Lost Time  by Marcel Proust
h4 2
. Ulysses by James Joyce
h4 3
. Don Quixote by Miguel de Cervantes
h4 4
. The Great Gatsby  by F. Scott Fitzgerald
h4 5
. One Hundred Years of Solitude  by Gabriel Garcia Marquez
h4 6
. Moby Dick by Herman Melville
h4 7
. War and Peace by Leo Tolstoy
h4 8
. Lolita  by Vladimir Nabokov
h4 9
. Hamlet by William Shakespeare
h4 10
. The Catcher in the Rye by J. D. Salinger
h4 11
. The Odyssey by Homer
h4 12
. The Brothers Karamazov  by Fyodor Dostoyevsky
h4 13
. Crime and Punishment  by Fyodor Dostoyevsky
h4 14
. Madame Bovary by Gustave Flaubert
h4 15
. The Divine Comedy  by Dante Alighieri
h4 16
. The Adventures of Huckleberry Finn by Mark Twain
h4 17
. Alice's Adventures in Wonderland  by Lewis Carroll
h4 18
. Pride and Prejudice by Jane Austen
h4 19
. Wuthering Heights by Emily Brontë
h4 20
. To the Lighthouse  by Virginia Woolf
h4 21
. Catch-22  by Joseph Heller
h4 22
. The Sound and the Fury by William Faulkner
h4 23
. Nineteen Eig

In [ ]:
response = urllib.request.urlopen(url)
results = json.load(response)


flattened_data = json_normalize(results)
flattened_data1 = json_normalize(flattened_data.articles[0])
flattened_data1

In [17]:
# Get the total number of pages
total_pages=results['pagination']['page_count']

# Your code
df=pd.DataFrame()
for i in range(total_pages):
    k=84*i
    url=f'https://dauntbooks.co.uk/book-categories/non-fiction/'
    response = urllib.request.urlopen(url)
    results = json.load(response)
    flattened_data = json_normalize(results)
    flattened_data1 = json_normalize(flattened_data.articles[0])
    flattened_data1=flattened_data1.set_index('sku')
    df = df.append(flattened_data1)
    print(f"getting page {i} of {total_pages}", end='\r')

df

NameError: name 'results' is not defined

In [18]:

df.brand_name.value_counts().index[0]

NameError: name 'df' is not defined

In [ ]:
#Our data is still text. Convert prices into numbers:
df['price.original']=df['price.original'].str.extract('(\d*,\d*)')
df['price.promotional']=df['price.promotional'].str.extract('(\d*,\d*)')

df['price.original'] = [x.replace(',', '.') for x in df['price.original']]
df['price.promotional'] = [x.replace(',', '.') for x in df['price.promotional']]

In [ ]:
df['discount_amount']=df['price.original'].astype(float)-df['price.promotional'].astype(float)
df1=df.copy()

In [ ]:
total_disc=df1.groupby(['brand_name']).sum().discount_amount

In [ ]:
total_disc.sort_values(ascending=False).index[0]

In [ ]:
#Display the brands without discount at all
total_disc[total_disc==0]